# Understand the prolongation and restriction operators

In [1]:
import sympy as sym
sym.init_printing()


h = sym.symbols('h')


def build_poisson_operator(n):
    A = sym.zeros(n,n)
    
    A[0,0] = 1
    A[n-1,n-1] = 1
    
    for i in range(1,n-1):
        A[i,i-1] = -1 / h**2
        A[i,i]   =  2 / h**2
        A[i,i+1] = -1 / h**2
        
    return A

In [2]:
build_poisson_operator(5)

⎡ 1    0    0    0    0 ⎤
⎢                       ⎥
⎢-1   2    -1           ⎥
⎢───  ──   ───   0    0 ⎥
⎢  2   2     2          ⎥
⎢ h   h     h           ⎥
⎢                       ⎥
⎢     -1   2    -1      ⎥
⎢ 0   ───  ──   ───   0 ⎥
⎢       2   2     2     ⎥
⎢      h   h     h      ⎥
⎢                       ⎥
⎢          -1   2    -1 ⎥
⎢ 0    0   ───  ──   ───⎥
⎢            2   2     2⎥
⎢           h   h     h ⎥
⎢                       ⎥
⎣ 0    0    0    0    1 ⎦

Use the inclusion to project onto the coarser grid. This strategy is easy but the corresponding prolongation operator is silly

In [3]:
def simple_projection(n):
    m = 1 + (n-1)//2
    R = sym.zeros(m,n)
    
    for i in range(m):
        R[i,2*i] = 1
        
    return R

In [4]:
simple_projection(5)

⎡1  0  0  0  0⎤
⎢             ⎥
⎢0  0  1  0  0⎥
⎢             ⎥
⎣0  0  0  0  1⎦

The restricted operator when applied the formula
$A^{2h} = RA^{h}R^{t}$

In [5]:
n = 9
A = build_poisson_operator(n)
R = simple_projection(n)

R @ A @ R.T

⎡1  0   0   0   0⎤
⎢                ⎥
⎢   2            ⎥
⎢0  ──  0   0   0⎥
⎢    2           ⎥
⎢   h            ⎥
⎢                ⎥
⎢       2        ⎥
⎢0  0   ──  0   0⎥
⎢        2       ⎥
⎢       h        ⎥
⎢                ⎥
⎢           2    ⎥
⎢0  0   0   ──  0⎥
⎢            2   ⎥
⎢           h    ⎥
⎢                ⎥
⎣0  0   0   0   1⎦

This feels wrong. It's like the simpler system is a Jacobi preconditioner of the original problem

## Starting from a good prolongation

In [6]:
def prolongation(m):
    n = 2 * (m - 1) + 1
    P = sym.zeros(n,m)
    
    for i in range(n):
        if i % 2 == 0:
            P[i,i//2] = 1
        else:
            P[i,i//2]     = sym.nsimplify(0.5)
            P[i,(i+1)//2] = sym.nsimplify(0.5)
            
    return P

In [7]:
prolongation(5)

⎡ 1    0    0    0    0 ⎤
⎢                       ⎥
⎢1/2  1/2   0    0    0 ⎥
⎢                       ⎥
⎢ 0    1    0    0    0 ⎥
⎢                       ⎥
⎢ 0   1/2  1/2   0    0 ⎥
⎢                       ⎥
⎢ 0    0    1    0    0 ⎥
⎢                       ⎥
⎢ 0    0   1/2  1/2   0 ⎥
⎢                       ⎥
⎢ 0    0    0    1    0 ⎥
⎢                       ⎥
⎢ 0    0    0   1/2  1/2⎥
⎢                       ⎥
⎣ 0    0    0    0    1 ⎦

In [8]:
P = prolongation(5)

P.T @ A @ P

⎡ 1     0     0     0     0  ⎤
⎢                            ⎥
⎢-1     1    -1              ⎥
⎢────   ──   ────   0     0  ⎥
⎢   2    2      2            ⎥
⎢2⋅h    h    2⋅h             ⎥
⎢                            ⎥
⎢      -1     1    -1        ⎥
⎢ 0    ────   ──   ────   0  ⎥
⎢         2    2      2      ⎥
⎢      2⋅h    h    2⋅h       ⎥
⎢                            ⎥
⎢            -1     1    -1  ⎥
⎢ 0     0    ────   ──   ────⎥
⎢               2    2      2⎥
⎢            2⋅h    h    2⋅h ⎥
⎢                            ⎥
⎣ 0     0     0     0     1  ⎦

Ok, this at least maintains the structure of the original system. Let's try to use the injection as the restriction and the just computed prolongation

In [9]:
R @ A @ P

⎡ 1     0     0     0     0  ⎤
⎢                            ⎥
⎢-1     1    -1              ⎥
⎢────   ──   ────   0     0  ⎥
⎢   2    2      2            ⎥
⎢2⋅h    h    2⋅h             ⎥
⎢                            ⎥
⎢      -1     1    -1        ⎥
⎢ 0    ────   ──   ────   0  ⎥
⎢         2    2      2      ⎥
⎢      2⋅h    h    2⋅h       ⎥
⎢                            ⎥
⎢            -1     1    -1  ⎥
⎢ 0     0    ────   ──   ────⎥
⎢               2    2      2⎥
⎢            2⋅h    h    2⋅h ⎥
⎢                            ⎥
⎣ 0     0     0     0     1  ⎦